In [9]:
import os

from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

chat = ChatOpenAI(
    temperature=0.1
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [11]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}")
])

prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)


AIMessage(content='Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')